## Traditional machine learning
Feature(s): 'SMILES' column

- Copolymer sequence of A and B monomer
- Binary encoding

Target: y_bin

- Binary class

In [1]:
import sys
import os

# Append the parent directory of your package to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', '..')))

In [2]:
import pandas as pd
import numpy as np
import zipfile

path_to_dataset = 'train_polymers.csv'
csv_filename = 'train_polymers.csv'

# Open the file, Correct the encoding and sep if necessary
if path_to_dataset.endswith('.zip'):
    with zipfile.ZipFile(path_to_dataset, 'r') as z:
        # Open the CSV file within the ZIP file
        with z.open(csv_filename) as f:
            # Read the CSV file into a DataFrame
            df = pd.read_csv(f, sep=',', on_bad_lines='warn')
else:
    # Read the CSV file into a DataFrame
    df = pd.read_csv(path_to_dataset, sep=',', on_bad_lines='warn')


print('Count of unique smiles:', df.SMILES.unique().shape[0])
print('Count of all of the smiles:', df.shape[0])


Count of unique smiles: 16000
Count of all of the smiles: 16000


In [3]:
# Define mapping
mapping = {'A': 0, 'B': 1}

# Apply mapping and convert sequences to arrays
def encode_sequence(sequence):
    return np.array([mapping[char] for char in sequence.split()])

df['Encoded'] = df['SMILES'].apply(encode_sequence)

In [4]:
from MLPipeline import MLmodel, BinTheTarget

Target = 'y_bin'
Features = ['Encoded']
Feature_types = ['numerical']
input = df[:7000]

/home/lsmo/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model = MLmodel(modelType='RandomForestClassifier',
                    df=input,
                    target=Target,
                    features=Features,
                    feature_types=Feature_types)

# get the values (input and output) of the model
X_train, X_test, y_train, y_test = model.getValues()

model.train()
model.evaluate()

RandomForestClassifier model trained successfully.
Accuracy for RandomForestClassifier: 0.88


0.88

In [6]:
from sklearn.model_selection import cross_val_score
from sklearn.base import clone

def objective(trial, model_instance):
    """
    Objective function for Optuna to minimize.
    """
    # Define hyperparameters to tune
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_categorical('max_depth', [None, 10, 20, 30, 40]),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 15),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 6),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False])
    }

    # Clone the model to ensure a fresh instance each trial
    model_clone = clone(model_instance.model)
    model_clone.set_params(**params)
    
    # Define the score metric
    scoring = 'accuracy'

    # Perform cross-validation
    scores = cross_val_score(model_clone, model_instance.X_train, model_instance.y_train, cv=model_instance.cv, scoring=scoring)

    # Return the average score across all folds
    return scores.mean()

In [7]:
model = MLmodel(modelType='RandomForestClassifier', df=df, target=Target, 
                features=Features, hyperparameter_tuning=True,
                feature_types=Feature_types,
                optimization_method='optuna', objective=lambda trial: objective(trial, model))

model.train()
predictions = model.predict()
model.evaluate()

[I 2024-09-11 19:04:37,082] A new study created in memory with name: no-name-cb6cf7f8-a0a5-4dda-9994-f7b75ae31224
[I 2024-09-11 19:04:37,613] Trial 0 finished with value: 0.76 and parameters: {'n_estimators': 131, 'max_depth': 40, 'min_samples_split': 4, 'min_samples_leaf': 6, 'max_features': 'log2', 'bootstrap': False}. Best is trial 0 with value: 0.76.
[I 2024-09-11 19:04:38,196] Trial 1 finished with value: 0.74 and parameters: {'n_estimators': 145, 'max_depth': 20, 'min_samples_split': 14, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'bootstrap': False}. Best is trial 0 with value: 0.76.
[I 2024-09-11 19:04:39,225] Trial 2 finished with value: 0.74 and parameters: {'n_estimators': 266, 'max_depth': 40, 'min_samples_split': 13, 'min_samples_leaf': 3, 'max_features': 'sqrt', 'bootstrap': False}. Best is trial 0 with value: 0.76.
[I 2024-09-11 19:04:40,416] Trial 3 finished with value: 0.76 and parameters: {'n_estimators': 203, 'max_depth': 10, 'min_samples_split': 11, 'min_samples_

Best RandomForestClassifier model trained successfully with hyperparameter tuning using Optuna.
Best hyperparameters: {'n_estimators': 113, 'max_depth': 30, 'min_samples_split': 5, 'min_samples_leaf': 5, 'max_features': 'log2', 'bootstrap': True}
RandomForestClassifier model trained successfully.
Accuracy for RandomForestClassifier: 0.78


0.78